In [2]:
import pandas as pd
import os
import mne
import numpy as np
import scipy
import matplotlib.pyplot as plt


filename_template =  "data/ds004504/{}/eeg/{}_task-eyesclosed_eeg.set"
# 1-36 have Alzheimers, 37-65 have frontotemporal dementia, 66-88 are healthy
ALZ_R = 36
FTD_R = 65
HEA_R = 88

H = 0
A = 1
F = 2

In [5]:
def read_raw_eeg_as_epochs(file, sampling_rate=120, n_channels=64):
    """
    Returns raw EEG epochs shaped (N, 1, 120, 64) for EEGNet.
    - Uses MNE's fixed-length epoching (no spectrograms).
    """
    import mne
    import numpy as np
    
    raw = mne.io.read_raw_eeglab(file, preload=True)
    raw.set_eeg_reference()
    
    raw.resample(sampling_rate)

    raw.pick_types(eeg=True)
    raw = raw.pick_channels(raw.ch_names[:n_channels])


    events = mne.make_fixed_length_events(raw, duration=2.0, overlap=1.0)
    epochs = mne.Epochs(
        raw,
        events,
        tmin=0,
        tmax=2.0,
        baseline=None,
        preload=True
    )

    X = epochs.get_data()   # shape: (E, 64, 240)

    epochs_resampled = epochs.copy().resample(120)
    X = epochs_resampled.get_data()  # (E, 64, 120)

    X = X.transpose(0, 2, 1)     # (E, 120, 64)
    X = X[:, None, :, :]         # (E, 1, 120, 64)

    return X.astype(np.float32)

In [6]:
def create_data():
    mne.set_log_level("WARNING")
    for sub in sorted([i for i in os.listdir("data/ds004504/derivatives")]):
        if "sub" not in sub:
            continue
        num = int("".join([j for j in filter(str.isdigit, sub)]))

        X = read_raw_eeg_as_epochs(filename_template.format(sub, sub))
        e = X.shape[0]

        label = -1
        if num <= ALZ_R:
            label = A
        elif num <= FTD_R:
            label = F
        else:
            label = H
        
        y = np.full(e, label)


        #Last variable means 1 second half window
        np.save(f"data/raw_eeg_data/{sub}_X.npy", X)
        np.save(f"data/raw_eeg_data/{sub}_y.npy", y)
        print(f"Saved {sub} data")
    mne.set_log_level("INFO")

create_data()
    

Saved sub-001 data
Saved sub-002 data
Saved sub-003 data
Saved sub-004 data
Saved sub-005 data
Saved sub-006 data
Saved sub-007 data
Saved sub-008 data
Saved sub-009 data
Saved sub-010 data
Saved sub-011 data
Saved sub-012 data
Saved sub-013 data
Saved sub-014 data
Saved sub-015 data
Saved sub-016 data
Saved sub-017 data
Saved sub-018 data
Saved sub-019 data
Saved sub-020 data
Saved sub-021 data
Saved sub-022 data
Saved sub-023 data
Saved sub-024 data
Saved sub-025 data
Saved sub-026 data
Saved sub-027 data
Saved sub-028 data
Saved sub-029 data
Saved sub-030 data
Saved sub-031 data
Saved sub-032 data
Saved sub-033 data
Saved sub-034 data
Saved sub-035 data
Saved sub-036 data
Saved sub-037 data
Saved sub-038 data
Saved sub-039 data
Saved sub-040 data
Saved sub-041 data
Saved sub-042 data
Saved sub-043 data
Saved sub-044 data
Saved sub-045 data
Saved sub-046 data
Saved sub-047 data
Saved sub-048 data
Saved sub-049 data
Saved sub-050 data
Saved sub-051 data
Saved sub-052 data
Saved sub-05